<big> 1. 假设有大量的(key,Value)键值对格式数据，希望找出按Value的排序的TOP N的数据，N为可变参数，程序应该把N作为一个变量处理。

 假设数据量很大，不能完全放在内存中排序，所以不能直接collect进行排序，也不能使用rdd的top()函数。<big>

In [52]:
d1 = sc.parallelize([('a',1), ('b',3), ('c', 20), ('d', 6), ('e', 9), ('f', 12), ('g', 5)], 4)

In [57]:
def genSeqOp(num):
    def op(dstList, data):
        if len(data) == 2: #把合法的数据放到结果中
            dstList.append(tuple(data))
        # 排个序，输出top N
        return sorted(dstList, key = lambda item: item[1], reverse=True)[:num]
    return op

def genCombOp(num):
    def op(dstList, data):
        print("dstList:{0}, data:{1}".format(dstList, data))
        dstList.extend(data) # 这里的data是每个分区的结果，也就是一个列表，所以用extend
        return sorted(dstList, key = lambda item: item[1], reverse=True)[:num]
    return op

In [58]:
 
d2 = d1.aggregate([], genSeqOp(2), genCombOp(2))
print(d2)

dstList:[], data:[('a', 1)]
dstList:[('a', 1)], data:[('c', 20), ('b', 3)]
dstList:[('c', 20), ('b', 3)], data:[('e', 9), ('d', 6)]
dstList:[('c', 20), ('e', 9)], data:[('f', 12), ('g', 5)]
[('c', 20), ('f', 12)]


<big>2.假设有大量的(Key,time,Value)格式数据，求出每个key的移动平均数。比如说我们有性能数据，key为objId+counterId，time为上报时间，value为性能值，

求出每个对象每个计数器的周移动平均数，不足一周的数据按前面所有的数据取平均。<big>

In [34]:
# 把每行数据按照 objid_conterId 为key的格式，拆成一个元素为tuple的list
def genCounter(row):
    objid = row[4]
    time = row[1]
    counters = row[7:13]
    rst = [("{0}_{1}".format(objid, i), (time, int(v))) for i, v in enumerate(counters) ]
    return rst

# 对每个 objid_conterId 下的计数器求移动平均
def movingAvg(datas):
    dataForWeek = []
    rst = []
    for data in datas:
        dataForWeek.append(data[1])
        if len(dataForWeek) > 7:
            dataForWeek = dataForWeek[1:]
        rst.append((data[0], sum(dataForWeek)/len(dataForWeek)))
    return rst
    
lines = spark.read.option("header", "true") \
             .csv("type7-1.csv").rdd \
             .flatMap(genCounter) \
             .groupByKey() \
             .mapValues(sorted) \
             .mapValues(movingAvg) \
             .collect()
print(lines)

[('33_3', [('2016-08-05 02:00:00', 3475.0), ('2016-08-05 02:15:00', 3470.0), ('2016-08-05 02:30:00', 3482.6666666666665)]), ('33_0', [('2016-08-05 02:00:00', 2901.0), ('2016-08-05 02:15:00', 2943.0), ('2016-08-05 02:30:00', 2916.3333333333335)]), ('33_2', [('2016-08-05 02:00:00', 2901.0), ('2016-08-05 02:15:00', 2953.0), ('2016-08-05 02:30:00', 2923.0)]), ('33_1', [('2016-08-05 02:00:00', 3475.0), ('2016-08-05 02:15:00', 3468.0), ('2016-08-05 02:30:00', 3481.3333333333335)]), ('33_4', [('2016-08-05 02:00:00', 2901.0), ('2016-08-05 02:15:00', 2932.5), ('2016-08-05 02:30:00', 2909.3333333333335)]), ('33_5', [('2016-08-05 02:00:00', 3475.0), ('2016-08-05 02:15:00', 3466.0), ('2016-08-05 02:30:00', 3480.0)])]
